# Lab 01 · FastAPI, Vite, and Git

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A FastAPI backend is scaffolded with health and echo routes.
- A Vite React frontend is outlined with modern tooling defaults.
- A Git repository is initialized with environment templates.

## What will be learned
- The structure of a basic FastAPI service is reviewed.
- Development-time CORS settings are configured.
- Vite-powered React scaffolding steps are rehearsed.
- Git initialization practices are reinforced.


### FastAPI framework primer

**FastAPI in a nutshell.** [FastAPI](https://fastapi.tiangolo.com/) is an asynchronous Python web framework that automatically maps HTTP paths to Python callables called *path operations*. Typed request and response models built with [Pydantic](https://docs.pydantic.dev/) provide validation, while async support keeps expensive AI inference calls from blocking the event loop.

#### Core building blocks
1. **Application entry point** — Create a `FastAPI()` instance in `app/main.py` and register startup/shutdown handlers to open or close model clients. [Docs: First Steps](https://fastapi.tiangolo.com/tutorial/first-steps/)
2. **Routers and path operations** — Organize endpoints in `routers/` modules with `APIRouter` to keep chat, embeddings, and admin surfaces separate. Mount each router in `main.py` to expose `/api/chat`, `/api/embeddings`, etc. [Docs: APIRouter](https://fastapi.tiangolo.com/tutorial/bigger-applications/)
3. **Pydantic models** — Declare request/response schemas in `schemas.py` so prompts, system messages, and model options are validated before invoking inference. [Docs: Request Body](https://fastapi.tiangolo.com/tutorial/body/)
4. **Dependency injection** — Use `Depends` to pass shared services (OpenAI client, vector store, cache) into route functions without manually instantiating them. This keeps AI integrations testable and configurable. [Docs: Dependencies](https://fastapi.tiangolo.com/tutorial/dependencies/)
5. **Background tasks** — Offload long-running work such as audit logging, metrics collection, or follow-up summarization via `BackgroundTasks`. Students can immediately see how to chain AI workflows beyond the initial response. [Docs: Background Tasks](https://fastapi.tiangolo.com/tutorial/background-tasks/)
6. **Interactive OpenAPI docs** — FastAPI generates `/docs` and `/redoc` from your type hints, letting you verify model contracts, stream responses, and share live documentation with teammates. [Docs: Interactive API docs](https://fastapi.tiangolo.com/features/#interactive-api-docs)

#### Tooling and project layout for AI routes
- **Local server:** Run `uvicorn app.main:app --reload` for fast feedback while editing routers or schemas. Enable hot reload so AI inference tweaks are visible immediately. [Docs: Uvicorn](https://fastapi.tiangolo.com/deployment/uvicorn/)
- **Environment management:** Load `.env` files with `python-dotenv` or FastAPI settings management to inject API keys, base URLs, and model IDs without hardcoding secrets.
- **Folder structure:** Start with `app/main.py`, `app/routers/`, `app/schemas.py`, and `app/services/ai.py`. Keep retry utilities, tracing helpers, and vector database adapters in `services/` so routes remain thin controllers.
- **Testing hooks (Optional):** Leverage FastAPI's `TestClient` to write unit tests that mock AI providers and confirm each router handles success, errors, and streaming payloads.

### Vite + React framework primer

#### Why Vite for React AI UIs
The frontend labs rely on [Vite](https://vitejs.dev/guide/) to scaffold a React UI with lightning-fast hot module replacement, native ES module bundling, and first-class TypeScript. CLI commands such as `npm create vite@latest`, `npm install`, `npm run dev`, and `npm run build` (or their `pnpm`/`yarn` equivalents) keep iteration tight while preparing optimized builds for deployment.

#### Core tooling and project conventions
- **Dev server workflow:** `npm run dev` launches a Vite server that can [proxy](https://vitejs.dev/config/server-options.html#server-proxy) `/api` calls to FastAPI, eliminating CORS friction while you develop AI features.
- **Environment variables:** Prefix keys with `VITE_` (e.g., `VITE_API_BASE_URL`, `VITE_OPENAI_MODEL`) so they are exposed via `import.meta.env`. Use separate `.env.local` files for experimental model toggles.
- **Module organization:** Mirror backend routers with feature folders like `src/features/chat/`, `src/features/embeddings/`, and keep shared helpers under `src/lib/` (`apiClient.js`, `retry.js`, `streamParser.js`).
- **React hooks and state:** Follow the [React documentation](https://react.dev/learn) to combine `useState`, `useEffect`, and custom hooks that orchestrate fetch lifecycles. Patterns such as `useReducer` for complex prompt builders or `useRef` for streaming buffers keep AI UIs maintainable.
- **UI composition:** Component libraries (Mantine, Chakra) or design systems can slot into `src/components/` for reusable chat bubbles, token counters, and loading spinners.

#### Mini walkthrough: bootstrapping the AI stack
1. **Scaffold FastAPI** with `python -m venv .venv && pip install fastapi uvicorn` and create `app/main.py` that instantiates the app, registers routers, and mounts static files if needed.
2. **Add routers and schemas** under `app/routers/inference.py` with `Pydantic` models describing your prompt payloads and inference responses.
3. **Wire dependencies** like model clients via `Depends`, storing credentials in `.env` and loading them with a settings class.
4. **Run `uvicorn app.main:app --reload`** to expose `/docs`, test path operations, and confirm background tasks fire during inference.
5. **Create the Vite React frontend** with `npm create vite@latest frontend -- --template react` and install UI dependencies that render form controls and streaming areas.
6. **Configure Vite proxy and envs** in `vite.config.js` and `.env.local` so `fetch` helpers in `src/lib/apiClient.js` point to FastAPI during development.
7. **Implement React hooks and components** (e.g., `useInference` in `src/features/chat/useInference.js`) that call the backend, manage retries, and render streaming updates.
8. **Iterate locally** by running `uvicorn` and `npm run dev` together, validating requests through the autogenerated FastAPI docs and the Vite dev tools before containerization or deployment.



## Prerequisites & install
The container workflow depends on these locally installed tools:

```bash
docker --version
docker compose version
git --version
```

After cloning the repository, build and start the services with Docker Compose:

```bash
cd ai-web
docker compose build
docker compose up
# Backend → http://localhost:8000
# Frontend → http://localhost:5173

# Shut the stack down when finished exploring:
docker compose down
```

## Step-by-step tasks
Each step configures local source files that the Docker stack will mount so secrets remain outside the built images.

### Step 1: Backend folder layout and Dockerfile
A backend folder is created with starter FastAPI files and a Dockerfile that installs dependencies inside the image.

### Step 1: Manually create the FastAPI backend scaffold

The backend now needs to be created manually so you can read through the starter code. Create the folders `ai-web/backend` and `ai-web/backend/app` if they do not already exist. Then, for each snippet below, copy the file path, create the file in your editor, and paste the commented code so you understand what every line does.

**File: `ai-web/backend/app/main.py`**
```python
from fastapi import FastAPI, HTTPException, Request  # Import FastAPI plus utilities for raising HTTP errors and inspecting requests.
from fastapi.middleware.cors import CORSMiddleware  # Bring in CORS middleware to allow the frontend to talk to this API.
from pydantic import BaseModel  # Import BaseModel to validate the request body for the echo endpoints.

app = FastAPI()  # Instantiate the FastAPI application object.
app.add_middleware(  # Register middleware that configures cross-origin requests.
    CORSMiddleware,  # Use the built-in CORS middleware implementation.
    allow_origins=["http://localhost:5173"],  # Allow the Vite dev server to call the API while running locally.
    allow_methods=["*"],  # Permit all HTTP methods so we are not artificially restricting traffic in the lab.
    allow_headers=["*"],  # Accept any headers, which keeps the demo simple.
)  # Close the add_middleware call.


_flaky_attempts: dict[str, int] = {}  # Track transient failure counts per client to simulate flaky behavior for retries.


class EchoIn(BaseModel):  # Define the expected request shape for the echo endpoints using Pydantic.
    msg: str  # Require a single string field called msg that the client will send.


@app.get("/health")  # Expose a health check route so Compose can verify the service is running.
def health():
    return {"status": "ok"}  # Return a simple JSON payload indicating that the API is healthy.


@app.post("/echo")  # Define the POST endpoint that will echo data back to the caller.
def echo(payload: EchoIn):
    return {"msg": payload.msg}  # Mirror the incoming message so the frontend can display it.


@app.post("/flaky-echo")  # Provide a variant that fails a configurable number of times before succeeding.
def flaky_echo(payload: EchoIn, request: Request, failures: int = 1):
    client_host = request.client.host if request.client else "unknown"  # Identify the caller so we can track failures per client.
    key = f"{client_host}:{failures}"  # Combine the host and failure target into a lookup key.
    seen = _flaky_attempts.get(key, 0)  # Read how many failures this client has already triggered.

    if seen < failures:  # If we have not yet hit the requested number of failures, raise an HTTP error.
        _flaky_attempts[key] = seen + 1  # Increment the failure counter for this client.
        raise HTTPException(status_code=503, detail="Simulated transient failure")  # Bubble up a 503 error that the frontend can retry.

    _flaky_attempts[key] = 0  # Reset the counter after the final failure so the next run starts fresh.
    return {"msg": payload.msg, "attempts": seen + 1}  # Return the echoed message plus how many attempts it took to succeed.
```

Explain in your own words how the flaky endpoint works: it intentionally throws a `503` until the requested number of failures has been observed, which lets the frontend exercise retry logic safely in development.

**File: `ai-web/backend/app/__init__.py`** (create this empty file to mark the package)
```python
# This file intentionally left blank so Python treats the backend app directory as a package.
```

**File: `ai-web/backend/.env.example`**
```env
# Document the required environment variables without checking real secrets into Git.
GEMINI_API_KEY=  # Replace this placeholder with your API key in a personal .env file when running the lab.
```

**File: `ai-web/backend/requirements.txt`**
```text
fastapi  # Core web framework that powers the backend API.
uvicorn[standard]  # ASGI server used to run FastAPI inside the Docker container.
pydantic  # Data validation library leveraged by FastAPI for request parsing.
python-dotenv  # Helper to load environment variables from .env files when developing locally.
google-generativeai  # Placeholder SDK dependency that later labs will use to call the Gemini API.
faiss-cpu  # Vector search library installed now so future labs can reuse the same environment.
numpy  # Numerical computing package required by FAISS and common ML workflows.
```

**File: `ai-web/backend/Dockerfile`**
```Dockerfile
# Use the official slim Python image as the container base to keep image size small.
FROM python:3.11-slim

# Declare /app as the working directory for subsequent instructions.
WORKDIR /app

# Set Python options to avoid writing .pyc files.
ENV PYTHONDONTWRITEBYTECODE=1

# Ensure Python flushes stdout/stderr immediately so logs appear in the terminal.
ENV PYTHONUNBUFFERED=1

# Copy the dependency list into the container so pip install can run.
COPY requirements.txt ./

# Install the Python dependencies declared in requirements.txt.
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application source code into the container image.
COPY app ./app

# Launch uvicorn when the container starts so the API becomes reachable.
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

After pasting these files into your project, save everything before moving to the frontend setup.



### Step 2: Frontend placeholders and Dockerfile
Frontend placeholders are positioned so Vite React components (`src/App.jsx`, `src/main.jsx`) can be customized while the generated `vite.config.js` continues to manage dev server defaults. A lightweight Dockerfile installs Node.js dependencies for the Vite dev server.

### Step 2: Build the React/Vite frontend by hand

Create the frontend files manually so you can trace how the UI talks to the backend. Copy the following snippets into the indicated files inside `ai-web/frontend`.

**File: `ai-web/frontend/src/lib/api.js`**
```javascript
const BASE = import.meta.env.VITE_API_BASE || "http://localhost:8000"; // Determine the backend base URL from Vite environment variables or fall back to localhost.

export async function post(path, body) { // Export a helper that wraps POST requests so components stay small.
  const res = await fetch(`${BASE}${path}`, { // Send the request to the composed URL using the Fetch API.
    method: 'POST', // Use POST because the echo endpoint expects JSON in the body.
    headers: { 'Content-Type': 'application/json' }, // Inform the server that the payload is JSON.
    body: JSON.stringify(body) // Serialize the payload object so FastAPI can parse it.
  });
  if (!res.ok) { // Check for non-2xx responses so we can raise a descriptive error.
    throw new Error(`HTTP ${res.status}`); // Throw an error containing the status code for easier debugging.
  }
  return res.json(); // Parse and return the JSON response body for the caller.
}
```

**File: `ai-web/frontend/src/App.jsx`**
```jsx
import { useState } from 'react'; // Pull in React's state hook so the component can manage form data.
import { post } from './lib/api'; // Import the API helper we just created for making POST requests.

function App() { // Declare the main application component rendered by Vite.
  const [msg, setMsg] = useState('hello'); // Track the message that the user wants to send to the backend.
  const [response, setResponse] = useState(''); // Store the echoed response text from the server.
  const [loading, setLoading] = useState(false); // Record whether an API request is in-flight to disable the button.
  const [error, setError] = useState(''); // Keep any error message produced when the API call fails.

  async function handleSend() { // Define the click handler that calls the backend.
    setLoading(true); // Indicate that the request has started so the UI can show a loading state.
    setError(''); // Clear any previous error before making a fresh request.
    try {
      const json = await post('/echo', { msg }); // Call the POST helper with the message payload.
      setResponse(json.msg); // Store the echoed message returned by FastAPI.
    } catch (err) {
      setError(String(err)); // Capture and display any errors that occur during the request.
    } finally {
      setLoading(false); // Always stop the loading indicator once the request finishes.
    }
  }

  return (
    <main style={{ padding: 24 }}> {/* Provide basic padding so the demo looks tidy. */}
      <h1>Lab 1 — Echo demo</h1> {/* Give the page a clear heading. */}
      <input value={msg} onChange={(event) => setMsg(event.target.value)} /> {/* Bind the input to state so the message updates as you type. */}
      <button onClick={handleSend} disabled={loading}>Send</button> {/* Call handleSend and disable the button while loading. */}
      {loading && <p>Loading…</p>} {/* Show a loading message when the API call is in progress. */}
      {error && <p style={{ color: 'red' }}>{error}</p>} {/* Render any errors in red so they stand out. */}
      <pre>{response}</pre> {/* Display the echoed message in a preformatted block. */}
    </main>
  );
}

export default App; // Export the component so main.jsx can render it.
```

**File: `ai-web/frontend/src/main.jsx`**
```jsx
import React from 'react'; // Import React to use JSX features.
import ReactDOM from 'react-dom/client'; // Grab the modern root API for rendering components.
import App from './App.jsx'; // Import the App component defined above.

ReactDOM.createRoot(document.getElementById('root')).render( // Create the root React tree inside the #root div.
  <React.StrictMode> // Wrap the app in StrictMode to highlight potential issues during development.
    <App /> // Render the App component as the sole child of the root.
  </React.StrictMode>,
); // Close out the render call with a trailing comma that Prettier will keep.
```

**File: `ai-web/frontend/vite.config.js`**
```javascript
import { defineConfig } from "vite"; // Import Vite's helper for building configuration objects.
import react from "@vitejs/plugin-react"; // Pull in the official React plugin so JSX works out of the box.

export default defineConfig({ // Export the configuration object that Vite will read.
  plugins: [react()], // Enable the React plugin to wire up Babel and Fast Refresh.
  server: { // Provide dev server options for local development.
    host: true, // Bind to all interfaces so Docker and local browsers can connect.
    port: 5173, // Match the default Vite port referenced in the backend CORS settings.
  },
});
```

**File: `ai-web/frontend/index.html`**
```html
<!doctype html> <!-- Declare the HTML document type for standards mode rendering. -->
<html lang="en"> <!-- Set the language attribute for accessibility and SEO. -->
  <head> <!-- Begin the head section containing metadata. -->
    <meta charset="UTF-8" /> <!-- Use UTF-8 encoding so all characters render correctly. -->
    <link rel="icon" type="image/svg+xml" href="/vite.svg" /> <!-- Reuse Vite's default favicon for the sample app. -->
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /> <!-- Ensure responsive sizing on mobile devices. -->
    <title>Lab 1 Echo Demo</title> <!-- Set the browser tab title. -->
  </head>
  <body> <!-- Begin the visible page content. -->
    <div id="root"></div> <!-- Placeholder div where React will mount the application. -->
    <script type="module" src="/src/main.jsx"></script> <!-- Load the React entry file using an ES module script. -->
  </body>
</html>
```

**File: `ai-web/frontend/package.json`**
```json
{
  "_comment1": "Name the package so npm can track it in the workspace.",
  "name": "frontend",
  "_comment2": "Version is arbitrary for the lab; 0.0.0 keeps things simple.",
  "version": "0.0.0",
  "_comment3": "Mark the project as private so npm refuses to publish it by accident.",
  "private": true,
  "_comment4": "Use ECMAScript modules because Vite expects them by default.",
  "type": "module",
  "_comment5": "Define scripts that drive the usual Vite workflows.",
  "scripts": {
    "_comment_dev": "Start the Vite development server with hot module reloading.",
    "dev": "vite",
    "_comment_build": "Compile the production build for deployment checks.",
    "build": "vite build",
    "_comment_preview": "Serve the production build locally to validate the output.",
    "preview": "vite preview --host"
  },
  "_comment6": "List runtime dependencies required by the frontend.",
  "dependencies": {
    "_comment_react": "React core library for building components.",
    "react": "^18.3.1",
    "_comment_react_dom": "DOM bindings that let React render into the browser.",
    "react-dom": "^18.3.1"
  },
  "_comment7": "List development-only tools.",
  "devDependencies": {
    "_comment_plugin": "Official Vite plugin that configures React JSX transformation.",
    "@vitejs/plugin-react": "^4.3.1",
    "_comment_vite": "Vite build tool that powers the dev server and bundler.",
    "vite": "^5.2.0"
  }
}
```

**File: `ai-web/frontend/Dockerfile`**
```Dockerfile
# Use the Node 20 alpine image for a lightweight frontend runtime.
FROM node:20-alpine

# Set the working directory where npm commands will execute.
WORKDIR /app

# Copy the manifest files first so npm install can leverage Docker layer caching.
COPY package*.json ./

# Install the JavaScript dependencies defined in package.json.
RUN npm install

# Copy the rest of the frontend source code into the container image.
COPY . .

# Start the Vite dev server when the container launches.
CMD ["npm", "run", "dev", "--", "--host"]
```

After adding these files, double-check that each directory matches the paths shown above.



### Step 3: Docker Compose orchestration
A top-level `docker-compose.yml` is created so the backend and frontend can be started together with a single command.

### Step 3: Define Docker Compose manually

Finally, create the Compose file yourself so you can see how the frontend and backend wire together. Copy the annotated YAML below into `ai-web/docker-compose.yml`.

```yaml
# Use Compose specification version 3.9 for compatibility with modern Docker installations.
version: "3.9"

# Declare the services that make up this project.
services:
  backend: # Define the FastAPI service configuration.
    build: ./backend # Build the backend image using the Dockerfile inside ai-web/backend.
    ports:
      - "8000:8000" # Map container port 8000 to localhost so the API is reachable from the browser.
    environment:
      - GEMINI_API_KEY=${GEMINI_API_KEY:-} # Pass through an optional Gemini API key for later labs.
    volumes:
      - ./backend/app:/app/app # Mount the backend source directory for live code edits.
      - ./backend/.env.example:/app/.env:ro # Provide a read-only example env file inside the container.

  frontend: # Define the React development server container.
    build: ./frontend # Build using the Dockerfile in ai-web/frontend.
    ports:
      - "5173:5173" # Expose the Vite dev server on the standard port.
    environment:
      - VITE_API_BASE=http://localhost:8000 # Tell the frontend how to reach the backend API.
    volumes:
      - ./frontend/src:/app/src # Mount source code for hot reloading during development.
      - ./frontend/vite.config.js:/app/vite.config.js # Keep configuration changes synced.
      - ./frontend/index.html:/app/index.html # Share the HTML entry point between host and container.
      - ./frontend/package.json:/app/package.json # Mount package.json to simplify dependency tweaks.
```

Once the file is saved, you can run `docker compose up` from inside `ai-web` when you're ready to launch both services.



### Step 4: Git initialization
Git is initialized locally so changes can be tracked.

### FastAPI ↔ Vite AI workflow checklist

1. Scaffold FastAPI with a dedicated router for AI inference (e.g., `routers/inference.py`) and register it in `main.py`.
2. Define `Pydantic` models for request payloads (prompt text, model options) and responses (structured outputs, tokens, metadata).
3. Configure dependency-injected clients for external AI providers, loading keys from `.env` files via `python-dotenv` or container secrets.
4. Enable CORS middleware and document the route via FastAPI's autogenerated docs to confirm payload contracts.
5. In Vite, create API utility modules that read backend URLs from `import.meta.env` and call the FastAPI endpoints.
6. Build React components that manage prompt forms, loading indicators, streaming updates, and error toasts tied to those utilities.
7. Use Vite's dev server proxy to forward `/api` requests to FastAPI for local iteration, then validate the flow through Docker Compose.
8. Update production Dockerfiles or CI pipelines so backend and frontend environments receive the same `.env` configuration for AI model credentials.


```bash
cd ai-web
git init
git add .
git commit -m "Lab 1 scaffold"
```

## Validation / acceptance checks
```bash
# locally
docker compose up -d
curl http://localhost:8000/health
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"hello"}'
docker compose down
```
- HTTP 200 responses include status "ok" and the echoed payload while the stack runs in Docker.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A README entry is expanded to document backend and frontend start commands.
- A GitHub repository is connected for remote backups.